In [46]:
import os
import re
from collections import defaultdict
from itertools import product
import pickle as pkl

import torch
import pandas as pd
import numpy as np

network_list = ['resnet18', 'vgg16', 'densenet121']
dataset_list = ['cifar10', 'gtsrb', 'imagenet']
method_list  = ['badnet', 'sig', 'ref', 'warp', 'imc', 'uap', 'ulp']

logdir = "/scr/songzhu/trojai/uapattack/result"
logs = os.listdir(logdir)

In [74]:
ASR_THRESH:float = 0.90
ACC_THRESH:float = 0.83

res_dict = defaultdict(list)

for combo in product(method_list, dataset_list, network_list):
    
    method, dataset, network = combo
    logfiles = list(filter(re.compile(f'({method}_{dataset}_{network})+.*').match, logs))
    
    for logfile in logfiles:
        
        with open(os.path.join(logdir, logfile), 'rb') as f:
            config = pkl.load(f)
            result = pkl.load(f)
        f.close()
        
        n_epochs = config['train'][config['args']['dataset']]['N_EPOCHS']
        
        res_dict['dataset'].append(dataset)
        res_dict['network'].append(network)
        res_dict['method'].append(method)
        
        res_dict['seed'].append(config['args']['seed'])
        res_dict['use_clip'].append(config['train']['USE_CLIP'])
        res_dict['use_transform'].append(config['train']['USE_TRANSFORM'])
        res_dict['use_advtrain'].append(config['adversarial']['ADV_TRAIN'])
        
        res_dict['acc'].append(max(result['test_clean_acc']))
        res_dict['asr'].append(max(result['test_troj_acc']))
        
        cond = (np.array(result['test_troj_acc']) >= ASR_THRESH) & (np.array(result['test_clean_acc']) >= ACC_THRESH)
        
        res_dict['T'].append(np.where(cond==True)[0].min() if sum(cond)>0 else n_epochs)

In [75]:
pd.DataFrame(res_dict).drop_duplicates()

,dataset,network,method,seed,use_clip,use_transform,use_advtrain,acc,asr,T
0,cifar10,resnet18,badnet,77,False,False,False,0.835316,0.999001,76
2,cifar10,vgg16,badnet,77,False,False,False,0.888811,0.999001,12
3,cifar10,resnet18,sig,77,False,False,False,0.862714,0.999001,15
4,cifar10,vgg16,sig,77,False,False,False,0.888611,0.999001,12
5,cifar10,resnet18,ref,77,False,False,False,0.833917,0.999001,68
6,cifar10,resnet18,warp,77,False,False,False,0.871775,0.990431,51
7,cifar10,vgg16,warp,77,False,False,False,0.898714,0.999719,37
